In [ ]:
from enum import Enum

class Piece(Enum):
    EMPTY = 0
    KING = 1
    DEFENDER = 2
    ATTACKER = 3

    def __str__(self):
        if self == Piece.EMPTY:
            return " "
        elif self == Piece.KING:
            return "K"
        elif self == Piece.ATTACKER:
            return "O"
        elif self == Piece.DEFENDER:
            return "X"
        else:
            raise ValueError("Invalid piece type")
        
class Coord():
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __eq__(self, other):
        if isinstance(other, Coord):
            return self.x == other.x and self.y == other.y
        return False

    x = 0
    y = 0

class game:
    board = []

    def __init__(self):
        self.current_player = None
        self.game_over = False
        self.board = []
        
    def fill_board_13_by_13(self):
        for x in range(13):
            row = []
            for y in range(13):
                if (x == 0 or y == 0 or x == 12 or y == 12) and (x in range(4, 9) or y in range(4, 9)):
                    row.append(Piece.ATTACKER)
                elif ((x == 1 or x == 11) and (y == 6)) or ((y == 1 or y == 11) and (x == 6)):
                    row.append(Piece.ATTACKER)
                elif (x == 6 and y == 6):
                    row.append(Piece.KING)
                elif ((x-6)**2 + (y-6)**2) ** 0.5 < 2:
                    row.append(Piece.DEFENDER)
                elif (x == 4 or x == 8) and y == 6:
                    row.append(Piece.DEFENDER)
                elif (y == 4 or y == 8) and x == 6:
                    row.append(Piece.DEFENDER)
                else:
                    row.append(Piece.EMPTY)
            self.board.append(row)

    def print_board(self):
        for row in self.board:
            print(" ".join(str(piece) for piece in row))
        print()

    def is_game_over(self):
        king_pos = None
        for i, row in enumerate(self.board):
            for j, piece in enumerate(row):
                if piece == Piece.KING:
                    king_pos = (i, j)
                    break
            if king_pos:
                break

        assert king_pos is not None, "King position not found on the board"

        x, y = king_pos
        board = self.board

        # Check if king is in a corner
        if (x, y) in [(0, 0), (0, 12), (12, 0), (12, 12)]:
            return True

        # Directions: up, down, left, right
        dirs = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        attacker_count = 0
        wall_count = 0

        for dx, dy in dirs:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 13 and 0 <= ny < 13:
                if board[nx][ny] == Piece.ATTACKER:
                    attacker_count += 1
            else:
                wall_count += 1

        # Surrounded by attackers on all four sides
        if attacker_count == 4:
            return True

        # Back against wall and surrounded by 3 attackers
        if wall_count == 1 and attacker_count == 3:
            return True

        return False
    
    def piece_at(self, pos: Coord):
        return self.board[pos.x][pos.y]
    
    def is_valid_move(self, from_pos: Coord, to_pos: Coord):
        assert all(0 <= element < 13 for element in [from_pos.x, from_pos.y, to_pos.x, to_pos.y]), "Coordinates must be between 0 and 12"

        if from_pos == to_pos:
            return False 
        
        piece = self.piece_at(from_pos)
        if piece == Piece.EMPTY:
            return False
        
        if from_pos.x == to_pos.x:
            row = self.board[from_pos.x]
            if from_pos.y < to_pos.y:
                leftmost = from_pos.y + 1
                rightmost = to_pos.y
            else:
                leftmost = to_pos.y
                rightmost = from_pos.y - 1

            for piece in row[leftmost:rightmost]:
                if piece != Piece.EMPTY:
                    return False                                                                                                                                                                                              
        
        elif from_pos.y == to_pos.y:
            if from_pos.x < to_pos.x:
                leftmost = from_pos.x + 1
                rightmost = to_pos.x
            else:
                leftmost = to_pos.x
                rightmost = from_pos.x - 1

            for piece in self.board[to_pos.x][leftmost:rightmost]:
                if piece != Piece.EMPTY:
                    return False   
        else:
            return False
        
        return True
    
    def move_piece(self, from_pos: Coord, to_pos: Coord):
        piece = self.piece_at(from_pos)
        self.board[to_pos.x][to_pos.y] = piece
        self.board[from_pos.x][from_pos.y] = Piece.EMPTY

    
instance = game()
instance.fill_board_13_by_13()
instance.print_board()

        O O O O O        
            O            
                         
                         
O           X           O
O         X X X         O
O O     X X K X X     O O
O         X X X         O
O           X           O
                         
                         
            O            
        O O O O O        



In [ ]:
def test_game_initialization():
    g = game()
    assert hasattr(g, 'board')
    assert isinstance(g.board, list)
    assert hasattr(g, 'is_game_over')

def test_fill_board_13_by_13_content():
    g = game()
    g.fill_board_13_by_13()
    assert len(g.board) == 13
    assert all(len(row) == 13 for row in g.board)
    # Check King position
    assert g.board[6][6] == Piece.KING
    # Check that attackers and defenders are present
    attackers = sum(piece == Piece.ATTACKER for row in g.board for piece in row)
    defenders = sum(piece == Piece.DEFENDER for row in g.board for piece in row)
    assert attackers == 24
    assert defenders == 12

def test_print_board_runs():
    g = game()
    g.fill_board_13_by_13()
    try:
        g.print_board()
    except Exception as e:
        assert False, f"print_board raised an exception: {e}"

def test_is_valid_move():
    g = game()
    g.fill_board_13_by_13()
    # Valid horizontal move for defender
    from_pos = Coord(6, 4)
    to_pos = Coord(6, 2)
    assert g.is_valid_move(from_pos, to_pos) is True, "Expected valid move for defender"

    # Invalid move: moving to same position
    from_pos.x, from_pos.y = 6, 5
    to_pos.x, to_pos.y = 6, 5
    assert g.is_valid_move(from_pos, to_pos) is False, "Expected invalid move to same position"

    # Invalid move: moving diagonally
    from_pos.x, from_pos.y = 6, 4
    to_pos.x, to_pos.y = 5, 3
    assert g.is_valid_move(from_pos, to_pos) is False, "Expected invalid diagonal move"

    # Invalid move: blocked by piece
    from_pos.x, from_pos.y = 6, 2
    to_pos.x, to_pos.y = 6, 8
    assert g.is_valid_move(from_pos, to_pos) is False, "Expected invalid move due to blocking piece"

def test_move_piece():
    g = game()
    g.fill_board_13_by_13()
    from_pos = Coord(6, 4)
    to_pos = Coord(6, 2)
    g.move_piece(from_pos, to_pos)
    assert g.piece_at(to_pos) == Piece.DEFENDER, "Expected piece to be moved to new position"
    assert g.piece_at(from_pos) == Piece.EMPTY, "Expected original position to be empty after move"


test_game_initialization()
test_fill_board_13_by_13_content()
test_print_board_runs()
test_is_valid_move()
test_move_piece()

        O O O O O        
            O            
                         
                         
O           X           O
O         X X X         O
O O     X X K X X     O O
O         X X X         O
O           X           O
                         
                         
            O            
        O O O O O        

